In [75]:
import pulp

def optimize_battery_charging(prices, capacity, charging_efficiency, discharging_efficiency, charging_power, discharging_power): #losses function can be included in parameters given to func
    
    num_periods = len(prices)

    # Create the LP problem instance
    prob = pulp.LpProblem("ChargeBattery", pulp.LpMaximize)

    # Define decision variables
    charge = pulp.LpVariable.dicts("Charge", range(num_periods), lowBound=0, upBound=charging_power)
    discharge = pulp.LpVariable.dicts("Discharge", range(num_periods), lowBound=0, upBound=discharging_power)
    soc = pulp.LpVariable.dicts("SOC", range(num_periods+1), lowBound=0, upBound=capacity)

    # Set the objective function
    prob += pulp.lpSum(prices[t] * discharge[t] - prices[t] * charge[t] for t in range(num_periods))
    
    # Set starting SOC
    prob += soc[0] == 0
    
    # Set constraints
    
    for t in range(num_periods):
        # Battery state update
        prob += soc[t+1] == soc[t] + charging_efficiency * charge[t] - discharge[t] * (1/discharging_efficiency)

        # Battery capacity constraints
        #prob += soc[t] <= capacity
        #prob += soc[t] >= 0
        
        # Power constraint
        #prob += charge[t] <= charging_power #charging_power[t] could include constraints from max. demand
        #prob += discharge[t] <= discharging_power
    
    # Solve the problem
    status = prob.solve(pulp.GLPK(msg = 0))

    # Print the solution status
    print("Status:", pulp.LpStatus[status])

    # Print the optimal values of variables
    for var in prob.variables():
        print(var.name, "=", var.varValue)

    # Print the optimal objective value
    print("Optimal value =", pulp.value(prob.objective))


    # Retrieve the optimal solution
    optimal_x = [pulp.value(charge[t]) for t in range(num_periods)]
    optimal_y = [pulp.value(discharge[t]) for t in range(num_periods)]

    return optimal_x, optimal_y, status

# Example usage
prices = [0.20, 0.13, 0.12, 0.18, 0.14]  # Electricity prices for each time step
capacity = 14 * 4  # Battery capacity
charging_efficiency = 0.95  # Charging efficiency (e.g., 90%)
discharging_efficiency = 0.95  # Discharging efficiency (e.g., 85%)
charging_power = 5
discharging_power = 5
# Define the losses function based on your specific charging and discharging losses calculation
def losses(x, y):
    # Example calculation: assume 2% losses per unit of electricity transferred
    return 0.02 * (x + y)

optimal_buy, optimal_sell, status = optimize_battery_charging(prices, capacity, charging_efficiency, discharging_efficiency, charging_power, discharging_power)

#print("Optimal Buy: ", optimal_buy)
#print("Optimal Sell: ", optimal_sell)


Status: Optimal
Charge_0 = 0.0
Charge_1 = 0.540166
Charge_2 = 5.0
Charge_3 = 0.0
Charge_4 = 0.0
Discharge_0 = 0.0
Discharge_1 = 0.0
Discharge_2 = 0.0
Discharge_3 = 5.0
Discharge_4 = 0.0
SOC_0 = 0.0
SOC_1 = 0.0
SOC_2 = 0.513158
SOC_3 = 5.26316
SOC_4 = 0.0
SOC_5 = 0.0
Optimal value = 0.22977841999999993


Status: Optimal
Charge_0 = 0.0
Charge_1 = 0.540166
Charge_2 = 5.0
Charge_3 = 0.0
Charge_4 = 0.0
Discharge_0 = 0.0
Discharge_1 = 0.0
Discharge_2 = 0.0
Discharge_3 = 5.0
Discharge_4 = 0.0
SOC_0 = 0.0
SOC_1 = 0.0
SOC_2 = 0.513158
SOC_3 = 5.26316
SOC_4 = 0.0
SOC_5 = 0.0
Optimal value = 0.22977841999999993

In [50]:
import pandas as pd
df = pd.read_pickle('data/final_dataframe.pkl')

In [55]:
price = df.iloc[:,4].to_list()

In [56]:
len(price)

163680

In [67]:
price1 = price[:30000]

In [60]:
len(price1)

100

In [69]:
df

,Date,Photovoltaics [MWh] Original resolutions,Photovoltaics [MW] Calculated resolutions,Total (grid load) [MWh] Original resolutions,Germany/Luxembourg [€/MWh] Calculated resolutions
0,2018-10-01 00:00:00,0.0,42805,NaN,59.53
1,2018-10-01 00:15:00,0.0,42805,NaN,59.53
2,2018-10-01 00:30:00,0.0,42805,NaN,59.53
3,2018-10-01 00:45:00,0.0,42805,NaN,59.53
4,2018-10-01 01:00:00,0.0,42805,"10,589.75",56.10
...,...,...,...,...,...
163675,2023-06-01 22:45:00,0,62579,12945.5,95.41
163676,2023-06-01 23:00:00,0,62579,12817.75,86.53
163677,2023-06-01 23:15:00,0,62579,12539.0,86.53
163678,2023-06-01 23:30:00,0,62579,12371.0,86.53


In [85]:
import pulp

def optimize_battery_charging(price, demand_after_solar, solar_surplus, capacity, charging_efficiency, discharging_efficiency, charging_power, discharging_power): #losses function can be included in parameters given to func
    #sense checks
    min(solar_surplus) >= 0
    min(demand_after_solar) >= 0

    num_periods = min(len(price),len(demand_after_solar),len(solar_surplus))
    
    # Create the LP problem instance
    prob = pulp.LpProblem("ChargeBattery", pulp.LpMaximize)

    # Define decision variables
    charge = pulp.LpVariable.dicts("Charge", range(num_periods), lowBound=0, upBound=charging_power)
    discharge = pulp.LpVariable.dicts("Discharge", range(num_periods), lowBound=0, upBound=discharging_power)
    soc = pulp.LpVariable.dicts("SOC", range(num_periods+1), lowBound=0, upBound=1)
    solar = pulp.LpVariable.dicts("Solar surplus", range(num_periods), lowBound=0)

    # Set the objective function
    prob += pulp.lpSum(price[t] * discharge[t] - price[t] * charge[t] for t in range(num_periods))
    #prob += pulp.lpSum(solar[t]*(1/(t+1)) for t in range(num_periods))
    # Set starting SOC
    prob += soc[0] == 0
    
    # Set constraints
    
    for t in range(num_periods):
        # Battery state update
        prob += soc[t+1] == soc[t] + (charging_efficiency * (charge[t] + solar[t]) - discharge[t] * (1/discharging_efficiency))*(1/(4 * capacity))

        # Solar constraint, i.e. cannot use more solar than what is available from the surplus
        prob += solar[t] <= solar_surplus[t]
        # Power constraint
        prob += solar[t] + charge[t] <= charging_power
        prob += discharge[t] <= demand_after_solar[t]
    # Solve the problem
    status = prob.solve(pulp.GLPK(msg = 0))

    # Print the solution status
    print("Status:", pulp.LpStatus[status])

    # Print the optimal values of variables
    for var in prob.variables():
        print(var.name, "=", var.varValue)

    # Print the optimal objective value
    print("Optimal value =", pulp.value(prob.objective))


    # Retrieve the optimal solution
    optimal_x = [pulp.value(charge[t]) for t in range(num_periods)]
    optimal_y = [pulp.value(discharge[t]) for t in range(num_periods)]

    return optimal_x, optimal_y, status

# Example usage
prices = [0.20, 0.13, 0.12, 0.18, 0.20]  # Electricity prices for each time step
demand_after_solar = [1, 2, 0, 4, 5]
solar_surplus = [0, 2, 1, 0, 0]
capacity = 14  # Battery capacity
charging_efficiency = 0.95  # Charging efficiency (e.g., 90%)
discharging_efficiency = 0.95  # Discharging efficiency (e.g., 85%)
charging_power = 5
discharging_power = 5
# Define the losses function based on your specific charging and discharging losses calculation
def losses(x, y):
    # Example calculation: assume 2% losses per unit of electricity transferred
    return 0.02 * (x + y)

optimal_buy, optimal_sell, status = optimize_battery_charging(prices, demand_after_solar, solar_surplus, capacity, charging_efficiency, discharging_efficiency, charging_power, discharging_power)

#print("Optimal Buy: ", optimal_buy)
#print("Optimal Sell: ", optimal_sell)


Status: Optimal
Charge_0 = 0.0
Charge_1 = 2.9723
Charge_2 = 4.0
Charge_3 = 0.0
Charge_4 = 0.0
Discharge_0 = 0.0
Discharge_1 = 0.0
Discharge_2 = 0.0
Discharge_3 = 4.0
Discharge_4 = 5.0
SOC_0 = 0.0
SOC_1 = 0.0
SOC_2 = 0.0843515
SOC_3 = 0.169173
SOC_4 = 0.093985
SOC_5 = 0.0
Solar_surplus_0 = 0.0
Solar_surplus_1 = 2.0
Solar_surplus_2 = 1.0
Solar_surplus_3 = 0.0
Solar_surplus_4 = 0.0
Optimal value = 0.8536009999999999
